In [1]:
#importing libraries
import cudf, cupy, cuml
from cuml.linear_model import LinearRegression

In [2]:
train = cudf.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = cudf.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [3]:
numerical_features = train.dtypes[train.dtypes != "object"].index
categorical_features = train.dtypes[train.dtypes == "object"].index

In [4]:
train.drop(categorical_features, axis = 1, inplace = True)
test.drop(categorical_features, axis = 1, inplace = True)

In [5]:
train.drop(['LotFrontage', 'MasVnrArea','GarageYrBlt'], axis = 1, inplace = True)
test.drop(['LotFrontage', 'MasVnrArea','GarageYrBlt'], axis = 1, inplace = True)

In [6]:
X = train.drop(['SalePrice'],axis=1)
Y = train['SalePrice']

In [7]:
from cuml.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

In [10]:
algorithm = ['svd', 'eig', 'svd-qr', 'svd-jacobi', 'qr']
sample_sub = cudf.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')
for i in algorithm:
    print("Algorithm:")
    print(i)
    lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = i)
    regressor = lr.fit(X_train.astype('float32'), y_train.astype('float32'))
    predicts = lr.predict(X_test)
    print("MSE:")
    print(cuml.metrics.regression.mean_squared_error(y_test.astype('int64'),predicts.astype('int64')))
    print("R2 Score:")
    print(cuml.metrics.regression.r2_score(y_test.astype('float32'), predicts.astype('float32')))
    print("MAE:")
    print(cuml.metrics.regression.mean_absolute_error(y_test.astype('int64'), predicts.astype('int64')))
    print('\n')
    sample_sub['SalePrice'] = predicts
    sample_sub.to_csv('submission_'+i+'.csv' , index = False)